More complex format
===================

* xml
* html

XML
===
    * data transfer
    * nested elements
    * document validation
    * parsers in most languages
    * focus on bussines
    * 

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

Html
====

```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [2]:
!pip install bs4

    100% |████████████████████████████████| 102kB 467kB/s a 0:00:01
You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [6]:
urls = [
    "https://www.wykop.pl/strona/{}".format(page)
    for page in range(1, 1000, 1)
]


In [7]:
import collections
from pprint import pprint as pp

def extract_wykop_detail(el):
    try:
        Wykop = collections.namedtuple('Wykop', ['count', 'title', 'desc', 'tags'])
        tags = map(lambda a: a.text, el.select('a.tag'))
        tags = filter(lambda tag: tag.startswith("#"), tags)
        tags = list(tags)
        x = Wykop(
            int(el.select('div.diggbox a span')[0].text),
            el.select('h2 a')[0]['title'],
            "desc",
            tags)
        return x
    except Exception as e:
        return None
    

In [8]:
from bs4 import BeautifulSoup as bs
import requests
from itertools import chain

pages = (requests.get(url) for url in  urls)
html_contents = (response.content for response in pages)
soups = (bs(html, 'html.parser') for html in html_contents)
items = (soup.select('#itemsStream .article') for soup in soups)
all_items_together = (chain.from_iterable(items))
extracted = (extract_wykop_detail(html_el) for html_el in all_items_together)
valid = (element for element in extracted if element is not None)







In [9]:
pp(next(valid))

Wykop(count=123, title='„Walili do nas ze wszystkiego”. Jak Amerykanie zmasakrowali Rosjan', desc='desc', tags=['#swiat', '#wojsko', '#rosja', '#sluzbyspecjalne', '#ciekawostki', '#syria'])


In [10]:
import itertools
first5 = [next(valid) for x in range(4)]

In [11]:
list(first5)

[Wykop(count=114, title='Skrajnie wychudzony orangutan uwięziony na łańcuchu. Z głodu stracił nawet włosy', desc='desc', tags=['#swiat', '#zwierzeta', '#ciekawostki', '#przyroda']),
 Wykop(count=232, title='Izrael już teraz pęka w szwach.  Co będzie za 30 lat gdy populacja się podwoi?', desc='desc', tags=['#swiat', '#zydzi', '#4konserwy', '#izrael']),
 Wykop(count=701, title='Znany "dryfter" rozwalił wynajete auto i nie chce placić!', desc='desc', tags=['#wroclaw', '#cars', '#auto', '#mercedes', '#bekazpodludzi']),
 Wykop(count=506, title="Jak 'patriotyczne' media wspierają polską gospodarkę", desc='desc', tags=['#polska', '#neuropa', '#4konserwy', '#1000urojenniezaleznychmediow', '#patriotyzmgospodarczy', '#afera'])]

In [12]:
import collections
from pprint import pprint as pp

Wykop = collections.namedtuple('Wykop', ['count', 'title', 'description', 'tags'])

def clear_text(text):
    return text.rstrip().replace("\n", '').replace("\t", '')

def grab_wykop_item_data(element):
    try:
        tags = map(lambda x: x.text, element.select('.article>div a.tag'))
        tags = filter(lambda x: x.startswith('#'), tags)
        
        return Wykop(
            int(element.select('.article>div a>span:first-child')[0].text),
            element.select('.article>div h2 a')[0]['title'],
            clear_text(element.select('.article>div .description')[0].text),
            list(tags))
    except Exception as e:
        return None

In [13]:
pages = (requests.get(url) for url in urls)
contents = (page.content for page in pages)
soups = (bs(content, 'html.parser') for content in contents)
lists_of_items = (soup.select('#itemsStream li.iC') for soup in soups)
all_items_together = chain.from_iterable(lists_of_items)
extracted = (grab_wykop_item_data(item) for item in all_items_together)
valid = (wykop for wykop in extracted if wykop is not None)

In [14]:
first5 = [next(valid) for x in range(400)]


In [15]:
list(tags)



NameError: name 'tags' is not defined

In [ ]:
import itertools
first5 = [next(valid) for x in range(4)]

In [ ]:
list(first5)

In [ ]:
tags = (wykop.tags for wykop in first5)

In [ ]:
all_tags = list(chain.from_iterable(tags))

In [ ]:
counter = {}

for tag in all_tags:
    if tag not in counter.keys():
        counter[tag] = 1
    else:
        counter[tag] = counter[tag] + 1

In [ ]:
counter

In [ ]:
import operator
list(reversed(sorted(counter.items(), key=lambda item: item[1])))

In [ ]:
from collections import Counter
c = Counter(all_tags)
c.most_common()

persistance
-------------

In [ ]:
import csv
with open('wykop.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in valid:
        writer.writerow(row)

In [ ]:
import json
with open('wykop.data', 'w') as destination_file:
    for row in valid:
        destination_file.write(json.dumps(row._asdict()) + "\n")

1. urls




In [16]:
urls = [
    "https://www.ceneo.pl/Pierscionki;0020-104-0-0-{}.htm".format(page)
    for page in range(1, 10, 1)
]

In [17]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain
from pprint import pprint as pp

def extract_details(el):
    try:
        return (el['data-id'], el['data-gaproductname'], int(el['data-price']))
    except Exception as e:
        return None

responses = (requests.get(url) for url in urls)
htmls = (r.content for r in responses)
soups = (bs(html, 'html.parser') for html in htmls)
items = (soup.select('div[data-price]') for soup in soups)
all_items_together = chain.from_iterable(items)
extracted = (extract_details(item) for item in all_items_together)
valid = (item for item in extracted if item is not None)


In [18]:
next(valid)

('da710de9-d663-4386-a584-0eb3bf1a80b2',
 '56895169/Durango Pierścionek Z Niebieskim Topazem I Cyrkoniami ',
 210)

In [19]:
import itertools
first100 = [next(valid) for x in range(200)]

In [20]:
def my_sort_element_key(item):
    return item[2]

ordered = reversed(sorted(first100, key=my_sort_element_key))

#vs

ordered = reversed(sorted(first100, key=lambda item: item[2]))

In [21]:
list(ordered)

[('bf4cfc21-92fe-41ab-94d9-f80feac41c4b',
  '56445597/Pierścionek z brylantem W059-585-B0.48',
  5600),
 ('8df600d4-daea-40e6-9e4c-5c8cf30925e8',
  '56445941/Pierścionek z brylantem W198-585-B0.42',
  4900),
 ('11c8d0e7-7d4b-4180-9085-3c1f2b01d670',
  '56032218/Marko Złoty Pierścionek Brylantami Obrotowe Koło Z Krzyżykiem Satelity ',
  4199),
 ('d9a18589-de67-4ddf-815f-0bfb15cf5636',
  '56445435/Pierścionek z brylantem W033-585-B0.30',
  3920),
 ('1c34b064-d356-43d6-bb5c-81faf3242fbb',
  '56445444/Pierścionek z brylantem W267-585-B0.30',
  3920),
 ('d3be5dbe-f706-4116-9d15-c4c982e1ee1d',
  '56445391/Pierścionek z brylantem W016-585-B0.33',
  3540),
 ('c631b4c1-b4a6-4d96-ba4a-68f52ed96616',
  '56032278/Marko Złoty Pierścionek Z Brylantem Dodatkowe Elementy Obrotowe Satelity ',
  2779),
 ('dfe0d345-f60c-405f-a3c5-8508bd336ed1',
  '56445971/Pierścionek z brylantem W215-585-CB0.09',
  2560),
 ('a611f76c-c5cb-483a-8b58-667106dfb62c',
  '56032212/Marko Złoty Pierścionek Z Brylantami Obrotowe